In [29]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import speech_recognition as sr
import pyttsx3
import openai
import requests
import json
from bs4 import BeautifulSoup as bs
from difflib import SequenceMatcher
import subprocess
import random
import datetime
import ssl
import certifi
import sys
import re


In [30]:
class person:
    name = ''
    def setName(self, name):
        self.name = name

In [31]:

openai.api_key_path = "../key.txt"
cur_model = "text-davinci-002"
steam_name="yusufwadi"
web= False

r3 = sr.Recognizer()
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice', voices[0].id)

imgs = ["image", "picture", "photo", "landscape", "painting", "drawing", "art"]
exits = ["quit","exit","close"]
person_obj = person()

In [32]:
# Create new automated instance of Brave
def runWeb():
    if web == False:
            # Define Brave path
        brave_path = "C:/Program Files/BraveSoftware/Brave-Browser-Nightly/Application/brave.exe"
        options = webdriver.ChromeOptions()
        options.binary_location = brave_path

        # set dl options
        prefs = {"download.default_directory": "C:/Users/thewa/Desktop/projects/computational_neuroscience/AI_ML/projects/mun_ching/crawlers/content"}

        options.add_experimental_option("prefs", prefs)
        options.add_experimental_option("detach", True)
        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        return driver
    else:
        body = driver.find_element("tag name", "body")
        body.send_keys(Keys.CONTROL + 't')

In [33]:
def callGPT(prompt,tokens,model=cur_model):

    completion = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=0,
        frequency_penalty=0.5,
        max_tokens=tokens,

    )

    response =" ".join(re.findall("[a-zA-Z]+", completion.choices[0].text))
    response = str(response)
    
    return response


In [34]:

def getGame(response):

    URL = "https://steamcommunity.com/id/" + steam_name + "/games?tab=all&xml=1"
    data = requests.get(url=URL)
    soup = bs(data.text, 'xml')
    names = soup.find_all("name")
    response = response.lower()

    for name in names:

        if ((SequenceMatcher(a=response, b=name.text.lower()).ratio()) > 0.70):
            engine.say(f"Now launching... {name.text}")
            engine.runAndWait()
            print(name.text)
            return name.parent.find("appID").text

    print("Game not found")

    # print("hello")
#getGame("no mans sky")


In [35]:
def smartSearch(listen=False,keyword=""):
    if listen==True:
        r3=sr.Recognizer()
        with sr.Microphone() as source:
            audio = r3.listen(source,phrase_time_limit=4)
            keyword = r3.recognize_google(audio)
    else:

        YOUR_PROMPT = f"Extract a search query from the following prompt: '{keyword}'"

        response = callGPT(YOUR_PROMPT,20)

        return response


In [36]:
def smartListen(command):
    r3=sr.Recognizer()
    with sr.Microphone() as source: 
        audio = r3.listen(source,phrase_time_limit=4)
        keyword = r3.recognize_google(audio)

        YOUR_PROMPT = " " + '"' + keyword + '"'

        response = callGPT(YOUR_PROMPT,20)
        print(response)
        return response


In [37]:
def intent(command):
    prompt = f"Extract a one word intent from the following prompt: '{command}'"
    response = callGPT(prompt,10)
    return response

In [38]:
def speak(audio_string):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice', voices[0].id)
    engine.say(audio_string)
    engine.runAndWait()

In [39]:

from time import time


def listening(tts=""):
    with sr.Microphone() as source:
        r3 = sr.Recognizer()
        r3.adjust_for_ambient_noise(source,duration=0.2)
        speak(tts)
        audio = r3.listen(source,timeout=5)
        voice=""
        try:
            voice= r3.recognize_google(audio)  # convert audio to text
        except sr.UnknownValueError: # error: recognizer does not understand
            print("...")
            
        except sr.RequestError:
            speak('Sorry, the service is down')
        print(f">> {voice.lower()}")
        return voice
        

In [40]:


def respond(command):
    
    r3 = sr.Recognizer()
    if command == "":
        return None
    else:
        c_intent = intent(command)
        print(c_intent)
    command = command.lower()
    c_intent = c_intent.lower()

    if c_intent == "search":
        if 'youtube' in command:

            response = smartSearch(keyword=command)
            say = f"Here is what I found for {response}"
            speak(say)
            driver = runWeb()
            driver.get(f'https://www.youtube.com/results?search_query={response}')
            driver.maximize_window()
            
        elif 'google' in command:
            driver =  runWeb()
            driver.get('https://www.google.com/')
            driver.maximize_window()        
            response = smartSearch()
        

            elem = driver.find_element('xpath', '//input[@class="gLFyf gsfi"]')
            elem.click()
            elem.send_keys(response, Keys.RETURN)

    #game
    elif c_intent == "play":

        print(command)

        prompt = f"Extract the name of the game from the prompt: {command}"

        response = callGPT(prompt,20)

        gameID = getGame(response)

        subprocess.call([r'../batch/steam.bat', gameID])

        # tryAudio(audio)
    # 1: greeting
    elif c_intent == "greeting":
        greetings = [f"hey, how can I help you {person_obj.name}", f"hey, what's up? {person_obj.name}", f"I'm listening {person_obj.name}", f"how can I help you? {person_obj.name}", f"hello {person_obj.name}"]
        greet = greetings[random.randint(0,len(greetings)-1)]
        speak(greet)

    # 2: name
    elif c_intent == "name" and "what" in command:
        if person_obj.name:
            speak("my name is Idaw")
        else:
            speak("my name is Idaw. what's your name?")

    elif c_intent == "name":
        person_name = command.split("is")[-1].strip()
        speak(f"okay, i will remember that {person_name}")
        person_obj.setName(person_name) # remember name in person object
    # 4: time
    elif c_intent == "time":
        e = datetime.datetime.now()
        speak("The time is now: = %s:%s" % (e.hour, e.minute))

    # 7: get stock price
    elif c_intent == "price":
        search_term = command.lower().split(" of ")[-1].strip() #strip removes whitespace after/before a term in string
        stocks = {
            "apple":"AAPL",
            "microsoft":"MSFT",
            "facebook":"FB",
            "tesla":"TSLA",
            "bitcoin":"BTC-USD"
        }
        try:
            stock = stocks[search_term]
            stock = yf.Ticker(stock)
            price = stock.info["regularMarketPrice"]

            speak(f'price of {search_term} is {price} {stock.info["currency"]} {person_obj.name}')
        except:
            speak('oops, something went wrong')
    elif c_intent == "exit" or c_intent == "stop":
        speak("going offline")
        sys.exit(0)

In [41]:
##      main loop

speak("What can I help with today?")
while(1):
        phrase=input("enter input: ")
        respond(phrase)
        



game
The intent from the word play
The intent from this prompt is play
The intent of the word exit


KeyboardInterrupt: Interrupted by user

# Voice Assist